# Problem 1: MCMC for Poisson-gamma model

Consider a model 
\begin{align*}
x_n &\sim \text{Poisson}(uv),\\
u &\sim \text{Gamma}(6, 1),\\
v &\sim \text{Gamma}(3, 3).
\end{align*}
where individual observations $x_n$ are conditionally independent of each other. Use MCMC to infer $p(u,v|\mathbf{x})$ for the set of observations $\mathbf{x}=[5, 3, 9, 13, 5, 3, 5, 1, 2, 7, 6, 5, 6, 7, 4]$

Solve the following tasks related to this model:

1. **Write down the joint log-probability** $\log p(\mathbf{x},u,v)$
2. **Derive a Gibbs sampler** for this model by providing the conditional distributions $p(u|v,\mathbf{x})$ and $p(v|u,\mathbf{x})$
3. **Implement the Gibbs sampler** you just derived
4. **Run both the Gibbs sampler and Metropolis sampler** (using the implementation provided in the lecture notebook, or some other) and inspect the results. Plot the marginal distributions and trace plots of $u$ and $v$ as well as a cross-plot of the posterior samples. Finally, report the mean and variance of $u$ and $v$.
5. **Compare Gibbs and Metropolis** in this task. Do you find the same posterior with both? Are there any differences?

In [7]:
x = np.array([5, 3, 9, 13, 5, 3, 5, 1, 2, 7, 6, 5, 6, 7, 4])

log_density(data, param):
    ...
    return value

# Before writing the Gibbs sampler you need pen-and-paper derivation of the conditionals

gibbs_sampler(data, initial_param, iterations):
    ...
    return samples

gibbs_samples = gibbs_sampler(x, ...)

metropolis_samples = metropolis_sampler(x, ...)

# Analyse results here
# (Can re-use code from the lecture notebook if you think it is useful, but think what you are plotting)

SyntaxError: invalid syntax (<ipython-input-7-c3ab3943ca0b>, line 3)

# Problem 2: Latent Dirichlet Allocation
    
Start by getting familiar with the LDA model by reading the material provided in Moodle.

The code snippet below generates data from the model. Make sure you understand the dimensionalities and meanings of all terms, so that you can correctly manipulate them in the samplers. We will later use the data to try out the sammplers.

The data is intentionally very small so that you do not need to worry about code efficiency. If your code is fast, feel free to try it with larger $N$, $V$ and/or $L$.

In [6]:
import numpy as np
from scipy import stats

np.random.seed(789789)

# Define the data dimensionalities
N = 40   # Number of documents
V = 100  # Vocabulary size
L = 50   # Length of each document (here identical for simplicity)

# Hyperparameters
alpha_true = 0.1
gamma_true = 0.1
K_true = 10

# Sample data from the model
def create_data(N, V, K, L):
    # Topic distributions of individual documents
    pi = stats.dirichlet(alpha_true*np.ones(K)).rvs(N)
    # Word distributions of individual topics
    b = stats.dirichlet(gamma_true*np.ones(V)).rvs(K_true)
    q = np.zeros((N, L), 'int')
    y = np.zeros((N, L), 'int')
    for n in range(N):
        for w in range(L):
            # Sample topic responsible for generating the word
            topic = np.random.choice(K_true, 1, p=pi[n,:])[0]
            q[n, w] = topic
            # Sample the word itself
            word = np.random.choice(V, 1, p=b[topic,:])[0]
            y[n, w] = word
    return y, pi, b, q

y, pi_true, b_true, q_true = create_data(N, V, K_true, L)

## (a) Direct Gibbs

Implement a direct Gibbs sampler, following Equations (27.30) - (27.32) that give the conditional distributions needed for sampling $\pi$ and $q_{il}$ (the responsible topic for each word) and $b_k$ (the topics). Note that here the book suddenly uses $x_{il}$ instead of $y_{il}$ -- this is a mistake in the book. 

For initialization (of both samplers) you can sample $\pi$ and $b$ from their priors and then use Equation (27.30) to get initial $q$.

In [8]:
# You will probably want to write a separate function for each conditional distribution

# You will also need a function that computes the $c_{ivk}$ based on $q_{il}$.
# Pay attention to the indexing; in $c_{ivk}$ the index $v$ is the word identity,
# which is *not* the same thing as $l$ that gives the running index of each word. 

# ...and then a simple loop for the main algorithm that uses the functions above

## (b) Collapsed Gibbs

Implement a collapsed Gibbs sampler. For this you will need the Equation (27.37) that provides the probabilities for sampling $q_{il}$ for one word conditional on the allocations for all others, integrating over $\pi$ and $b$. Note that you can still use equations (27.31) and (27.32) to obtain $\pi$ and $b$ for interpretation after running the algorithm.

Collapsed Gibbs is easiest to implement by storing and updating both $c_{ivk}$ and $q_{il}$. To allocate a new sample you first subtract one from $c_{ivk}$ based on current value $q_{il}$ before computing all the sums for equation (27.37). After sampling a new value you add one to the right element and update the corresponding element in $q_{il}$. This is inefficient compared to explicitly maintaining also the different marginal sums, but does not matter for our small data.

In [ ]:
# Implement here the collapsed sampler

# (c) Evaluate the samplers

Run both samplers on the toy data and study the results. You can use the same $\alpha$, $\gamma$ and $K$ that were used when creating the data.

1. Plot the "document-to-topic" and "topic-to-word" distributions for the last posterior sample and compare them against the true values used when generating the data. Note that LDA is not permutation-invariant -- there is no reason to expect the topics would appear in the same order in your solution as in the generation process. To better compare them, you can sort the topics according to their overall popularity.

2. Investigate a single document to understand how the model works. Check that the document is using topics that have high probability of generating the words that appear in the document. Try to be 'properly Bayesian' here, in the sense that you do not look only at the last posterior sample but attempt to characterise also the variation in some way.

3. Would you prefer the direct or collapsed sampler? Was one of them easier to implement? Do you think one works better?

